In [50]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
sns.set_style('white')
%matplotlib inline
from surprise import Dataset, Reader
from surprise import SVD
from surprise import accuracy
from surprise.model_selection import cross_validate, train_test_split
from surprise.prediction_algorithms import knns
from surprise.model_selection import train_test_split
from surprise import Reader, Dataset
#prediction model imports
from surprise.model_selection import cross_validate
from surprise.prediction_algorithms import SVD
from surprise.prediction_algorithms import KNNWithMeans, KNNBasic, KNNBaseline
from surprise.model_selection import GridSearchCV
# If you want to compare item-item
from surprise.similarities import cosine, msd, pearson

In [51]:
#import data
df = pd.read_csv('df.csv', index_col=0)
df2 = pd.read_csv('df2.csv', index_col=0)
df3 = pd.read_csv('df3.csv', index_col=0)

# Train Test Split Data

In [52]:
# scale the rating from 1-10 .... default is 1-5
reader = Reader(rating_scale=(1, 10))
book_list = list(df2['title'])

In [54]:
# for trainset, testset the data needs to be in object form 
data = Dataset.load_from_df(df2[['user_id','title','book_rating']], reader)
trainset, testset = train_test_split(data, test_size = 0.30, random_state=42)

# 46,585 user ratings 
print(df2.shape)
print(df3.shape)

(46585, 3)
(46585, 10)


# Utility Matrix

In [49]:
book_matrix = df3.pivot_table(index='user_id', columns='title', values='book_rating')
book_matrix.head()

title,Dark Justice,Flight Of Fancy American Heiresses Zebra Ballad Romance,Microsoft Application Architecture For Microsoft .Net Designing Application Pro-Other,Murder Of A Sleeping Beauty Scumble River Mysteries Paperback,$30 Film School,'A Hell Of A Place To Lose A Cow' An American Hitchhiking Odyssey,'Round The Corner Sister Circle,'Til Death Do Us Part,'Tis Herself A Memoir,10 Minutes Till Bedtime,...,Zipping My Fly Moments In The Life Of An American Sportsman,Zippy Annual 2001,Zobmondo The Outrageous Book Of Bizarre Choices,Zodiac,Zoya'S Story An Afghan Woman'S Struggle For Freedom,Zulu Heart,"\But Don'T All Religions Lead To God?\""""","\O\"" Is For Outlaw""","\The Gold Club\"" The Jacklyn \""Diva\"" Bush Story How I Went From Gold Room To Court Room""","\You'Ll Never Nanny In This Town Again!\"" The Adventures And Misadventures Of A Hollywood Nanny"""
user_id,,,,,,,,,,,,,,,,,,,,,
42,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
114,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
129,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# KNN Basic 

In [55]:
knn_basic = KNNBasic(sim_options={'name':'pearson','user_based':True})
cv_knn_basic= cross_validate(knn_basic,data,n_jobs=-1)
for i in cv_knn_basic.items():
    print(i)
print('-----------------------')
print(np.mean(cv_knn_basic['test_rmse']))

('test_rmse', array([1.8274292 , 1.8260086 , 1.82718515, 1.8509944 , 1.84524241]))
('test_mae', array([1.41061019, 1.41191602, 1.41600529, 1.42578742, 1.4353661 ]))
('fit_time', (13.701164960861206, 13.209265947341919, 12.506980180740356, 11.474082231521606, 9.678462028503418))
('test_time', (0.48518800735473633, 0.3675363063812256, 0.2923400402069092, 0.2628309726715088, 0.2317030429840088))
-----------------------
1.8353719528060786


#  KNN - Baseline

In [56]:
knn_baseline = KNNBaseline(sim_options={'name':'pearson','user_based':True})
cv_knn_baseline = cross_validate(knn_baseline,data)
for i in cv_knn_baseline.items():
    print(i)
np.mean(cv_knn_baseline['test_rmse'])

Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
('test_rmse', array([1.62386682, 1.63661521, 1.67651737, 1.65102207, 1.64433708]))
('test_mae', array([1.24922391, 1.24446485, 1.27220311, 1.26133487, 1.25633722]))
('fit_time', (4.320934057235718, 4.51840615272522, 4.389546871185303, 4.344373941421509, 4.193351984024048))
('test_time', (0.21333599090576172, 0.4168550968170166, 0.18823790550231934, 0.20849180221557617, 0.19227910041809082))


1.646471709800418

# Perform SVD

In [7]:
svd = SVD()
svd.fit(trainset)
predictions = svd.test(testset)
accuracy.rmse(predictions)

RMSE: 1.5905


1.5905365241889926

- **best performing model**

# Fitting and evaluating models

In [8]:
final = SVD(n_epochs=20, n_factors=100, biased=True, 
              lr_all=0.005, reg_all=0.1)
final.fit(trainset)
predictions = final.test(testset)
accuracy.rmse(predictions)
accuracy.mae(predictions)

RMSE: 1.5880
MAE:  1.2137


1.2137145313374114

# Recommendations for Existing User

In [9]:
def get_recs(user, df):
    """Returns top 5 recommended books based on predicted rating"""
    
    user_score_pred = []
    for i in df['title']:
        if svd.predict(user,i)[2] == None:
            user_score_pred.append(svd.predict(user,i))
        else:
            continue
    
    reccomendations = []
    for i in user_score_pred:
        if i[3]>=3:
            reccomendations.append(i)
    
    rec_dict = {}
    for i in reccomendations:
        rec_dict.update({i[1]: i[3]})
    print('Top 10 Recommendations:')
    return sorted(rec_dict.items() ,  key=lambda x: x[1], reverse=True)[:10]

get_recs(42, df3)

Top 10 Recommendations:


[('Harry Potter And The Prisoner Of Azkaban  Book 3  ', 9.084677578129675),
 ('Harry Potter And The Chamber Of Secrets Postcard Book ', 8.954611194482315),
 ('Harry Potter And The Order Of The Phoenix  Book 5  ', 8.922235110689702),
 ("East Of Eden  Oprah'S Book Club  ", 8.867278672266039),
 ('Harry Potter And The Goblet Of Fire  Book 4  ', 8.793537884070927),
 ('The Secret Life Of Bees ', 8.77916881778577),
 ('Speak ', 8.769647846591928),
 ('The Godfather ', 8.76398751768827),
 ('Lost In A Good Book  Thursday Next Novels  Penguin Books   ',
  8.645076448265637),
 ('Sisterhood Of The Traveling Pants ', 8.638027520155488)]

# Making predictions on a new user

In [12]:
def book_rater(df,num, author=None):
    """
    The function book_rater should take as parameters:
            df : DataFrame - a dataframe containing the movie ids, name of movie, and genres
            num : int - number of ratings to enter! 
            author : string - a specific author from which to draw movies
    The function returns:
            rating_list : list - a collection of dictionaries in the format of {'userId': int , 'movieId': int ,'rating': float }
    """
    userID = 1
    rating_list = []
    while num > 0:
        if author:
            book = df[df['author'].str.contains(author)].sample(1)
        else:
            book = df.sample(1)
        print(book)
        book_rating = input('How do you rate this book on a scale of 1-10, press n if you have not seen :\n')
        if book_rating == 'n':
            continue
        else:
            rating_one_book = {'user_id':userID,'title':book['title'].values[0],'book_rating':book_rating}
            rating_list.append(rating_one_book) 
            num -= 1

    return rating_list

In [13]:
#New User will rate 5 Nora Roberts books (enter in ratings)
user_rating = book_rater(df3, 5,'Nora Roberts')

       user_id      city        state country  age        isbn  book_rating  \
80824   227520  San Jose   California     Usa   33  051512317X            9   

             author   publisher          title  
80824  Nora Roberts  Jove Books  Rising Tides   
How do you rate this book on a scale of 1-10, press n if you have not seen :
6
       user_id         city           state country  age       isbn  \
10961    29888  Hedgesville   West Virginia     Usa   65  373285000   

       book_rating        author   publisher  \
10961            9  Nora Roberts  Silhouette   

                                       title  
10961  Captivated  Silhouette Single Title    
How do you rate this book on a scale of 1-10, press n if you have not seen :
9
       user_id            city      state country  age       isbn  \
43822   114368  Mechanicsville   Maryland     Usa   33  373218192   

       book_rating        author   publisher                title  
43822           10  Nora Roberts  Silhouette

In [15]:
# add new user to dataframe 
new_ratings_df = df3.append(user_rating,ignore_index=True)
new_data = Dataset.load_from_df(new_ratings_df[['user_id','title','book_rating']],reader)
svd_new = SVD()
svd_new.fit(new_data.build_full_trainset())

In [16]:
#view new df with new user
new_ratings_df.tail()

,user_id,city,state,country,age,isbn,book_rating,author,publisher,title
46585,1,NaN,NaN,NaN,NaN,NaN,6,NaN,NaN,Rising Tides
46586,1,NaN,NaN,NaN,NaN,NaN,9,NaN,NaN,Captivated Silhouette Single Title
46587,1,NaN,NaN,NaN,NaN,NaN,8,NaN,NaN,Engaging The Enemy
46588,1,NaN,NaN,NaN,NaN,NaN,9,NaN,NaN,Face The Fire Three Sisters Island Trilogy
46589,1,NaN,NaN,NaN,NaN,NaN,10,NaN,NaN,Key Of Light Key Trilogy Paperback


In [17]:
#Recommendations for new user
get_recs(1, new_ratings_df)

Top 10 Recommendations:


[('Harry Potter And The Chamber Of Secrets Postcard Book ', 9.108392991421356),
 ('Harry Potter And The Prisoner Of Azkaban  Book 3  ', 9.055137679150263),
 ('Harry Potter And The Order Of The Phoenix  Book 5  ', 8.982467304275069),
 ("Harry Potter And The Sorcerer'S Stone  Book 1  ", 8.844156492410134),
 ("East Of Eden  Oprah'S Book Club  ", 8.832216916689015),
 ('Speak ', 8.828390960632555),
 ('Harry Potter And The Goblet Of Fire  Book 4  ', 8.819897639756958),
 ("The Time Traveler'S Wife ", 8.817812615788162),
 ('Stiff The Curious Lives Of Human Cadavers ', 8.810708869996972),
 ('The Lord Of The Rings  Movie Art Cover  ', 8.774431339828071)]

# Item to Item 

In [58]:
#tried 2 similarity metrics.... 
sim_pear = {'name':'pearson','user_based':False}
sim_cos = {'name':'cosine','user_based':False}

#fit the model using KNN 
basic = knns.KNNBasic(sim_options=sim_cos)
basic.fit(trainset)
predictions_knn = basic.test(testset)
print(accuracy.rmse(predictions_knn))

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 1.7777
1.7777209507991454


In [21]:
#test the basic predict
basic.predict(98391, 'The Lovely Bones A Novel')

Prediction(uid=98391, iid='The Lovely Bones A Novel', r_ui=None, est=7.871201202122114, details={'was_impossible': True, 'reason': 'User and/or item is unkown.'})

In [59]:
def bookrec(book, k=5):
    """
    This function takes in a name of a book and gives you k recommendations for other books that you may like
    """
    iid = basic.trainset.to_inner_iid(book) #changes the name of the book to its id
    recs = basic.get_neighbors(iid, k) #takes the id and gives you k recommendations for it (also in id form)
    for rec in recs:
        print(basic.trainset.to_raw_iid(rec)) #takes the id and converts that back into a beer name and prints it

In [60]:
#Enter book and result will output the top 5 recommendations
bookrec('The Five People You Meet In Heaven ')

The Empire Of Shadows 
Bee Season A Novel 
The Consignment 
Self Matters  Creating Your Life From The Inside Out 
Basket Case 
